In [1]:
# Establish some constants

from pathlib import Path

DATA_FILE = Path("data/raw_data.jsonl")

In [2]:
# Get current ledger

import json
import requests

if DATA_FILE.exists():
    with open(DATA_FILE, 'r') as file:
        for line in file:
            pass
        obj = json.loads(line)
        current_ledger = obj['ledger_index'] - 1
else:
    response = requests.get("https://api.xrpscan.com/api/v1/ledgers").json()
    current_ledger = response['current_ledger']

print(f"Starting from ledger {current_ledger}")

Starting from ledger 94070582


In [ ]:
# Get the ledgers
while True:
    # Get a new ledger, breaking if xrpscan refused the request
    ledger_response = requests.get(f"https://api.xrpscan.com/api/v1/ledger/{current_ledger}?origin=mlabs.city")
    current_ledger -= 1
    code = ledger_response.status_code
    if code == 500:
        print(f"Internal server error when fetching ledger {current_ledger + 1}")
        continue
    if code != 200:
        print(f"Received response code of {code} for ledger {current_ledger + 1}")
        break

    # Record the ledger object
    ledger = ledger_response.json()
    with open(DATA_FILE, 'a') as file:
        file.write(json.dumps(ledger))
        file.write("\n")